<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Gemma_7B_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Dependencies:**

In [1]:
!pip install -qU bitsandbytes
!pip install -qU trl
!pip install -qU transformers
!pip install -qU peft
!pip install -qU optimum
!pip install -qU datasets
!pip install -qU accelerate
!pip install -qU nltk
!pip install -qU rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
from datasets import load_dataset
dataset = load_dataset("kaifahmad/indian-history-hindi-QA-3.4k",split="train")

In [11]:
dataset

Dataset({
    features: ['Question', 'Answer'],
    num_rows: 3468
})

In [12]:
def process_data_sample(example):
    # Extract relevant information from the example
    question = example['Question']
    answer = example['Answer']
    # Prepare the processed example for a History Question Answering System
    processed_example = (
        "आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। "
        "आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।\n\n"
        f"उपयोगकर्ता प्रश्न:\n{question}\n\n"
        f"उत्तर:\n{answer}\n\n"
    )
    return processed_example

In [13]:
dataset = dataset.map(lambda example: {'text': process_data_sample(example)}, remove_columns=['Question', 'Answer'])

In [14]:
for i in range(5):
    print(dataset[i]['text'])

आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।

उपयोगकर्ता प्रश्न:
प्राचीन मानवों के भारतीय उपमहाद्वीप में प्रवास के प्रमुख सम्भावित मार्ग क्या थे?

उत्तर:
प्राचीन मानवों का भारतीय उपमहाद्वीप में प्रवास संभवतः अरब सागर और बंगाल की खाड़ी के समुद्रतटीय मार्गों के साथ ही उत्तर-पश्चिमी प्रवासी द्वार के माध्यम से भी हो सकता था।


आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।

उपयोगकर्ता प्रश्न:
प्राचीन मानवों को भारतीय उपमहाद्वीप में प्रवास करने के लिए कौन-कौन से कारक प्रेरित कर सकते थे?

उत्तर:
प्राचीन मानवों का भारतीय उपमहाद्वीप में प्रवास विभिन्न कारकों से प्रेरित हो सकता था, जैसे कि पर्यावरणीय परिवर्तन, नए संसा

In [15]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [16]:
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline,
    logging
)
from sklearn.model_selection import train_test_split
from datasets import Dataset
from huggingface_hub import notebook_login
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

**Model Loading and Quantization:**

In [17]:
model_name = 'google/gemma-2b'

In [18]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [19]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [20]:
# Tokenization and Padding
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [21]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [22]:
# LoRA Config
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias='none',
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [23]:
# Training Arguments
training_arguments = TrainingArguments(
    output_dir='Finetuned-Model',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    save_strategy='epoch',
    logging_steps=10,
    save_steps=10,
    num_train_epochs=5,
    max_steps=50,
    fp16=True,
    warmup_ratio=0.05,
    push_to_hub=False,
)

In [24]:
import datasets
train_dataset = datasets.Dataset.from_dict(train_data)
test_dataset = datasets.Dataset.from_dict(test_data)

# SFTTrainer Arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=264
)

Map:   0%|          | 0/2774 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.079300
20,1.722300
30,1.475900
40,1.304100
50,1.202700


TrainOutput(global_step=50, training_loss=1.556857738494873, metrics={'train_runtime': 256.5287, 'train_samples_per_second': 3.119, 'train_steps_per_second': 0.195, 'total_flos': 1951127133683712.0, 'train_loss': 1.556857738494873, 'epoch': 0.29})

In [26]:
output_model_dir = "Gemma-7B-finetuned-indian-history"
trainer.model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

('Gemma-7B-finetuned-indian-history/tokenizer_config.json',
 'Gemma-7B-finetuned-indian-history/special_tokens_map.json',
 'Gemma-7B-finetuned-indian-history/tokenizer.model',
 'Gemma-7B-finetuned-indian-history/added_tokens.json',
 'Gemma-7B-finetuned-indian-history/tokenizer.json')

In [27]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

# Load the finetuned model and tokenizer
finetuned_model = AutoModelForCausalLM.from_pretrained(output_model_dir)
tokenizer = AutoTokenizer.from_pretrained(output_model_dir)

# Function to generate responses
def generate_response(context):
    input_ids = tokenizer.encode(context, return_tensors="pt")
    output = finetuned_model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)


test_samples = test_data['text']
generated_responses = [generate_response(sample) for sample in test_samples]
reference_responses = test_data['text']

bleu_score = corpus_bleu([[r] for r in reference_responses], generated_responses)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(generated_responses, reference_responses)

print("BLEU Score:", bleu_score)
print("ROUGE Scores:", rouge_scores)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


ValueError: Input length of input_ids is 164, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.